In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pprint import pprint

# 출력 구조 정의 (여행지, 기간, 예산, 평점, 활동)
response_schemas = [
    ResponseSchema(name="destination", description="여행한 장소 또는 도시명"),
    ResponseSchema(name="duration", description="여행 기간 (예: '2박 3일')"),
    ResponseSchema(name="budget", description="전체 여행 예산 또는 비용 (예: '30만원')"),
    ResponseSchema(name="rating", description="5점 만점에서 추천도 (숫자 1~5 중 하나)"),
    ResponseSchema(name="activities", description="주요 활동 목록 (문자열 리스트)")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print("출력 형식 지시사항:")
print(format_instructions)

# 프롬프트 템플릿
template = """
다음 여행 후기를 분석해 아래 정보를 추출하세요.

여행 후기:
{review}

{format_instructions}
"""

# 프롬프트 구성
prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5
)

# 테스트 입력 (여행 후기)
review = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 100만원 정도 썼는데 해운대에서 바다구경하고, 
자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 
5점 만점에 4점 정도 줄 수 있을 것 같아요.
"""

# 체인 실행
chain = prompt | model | parser
output = chain.invoke({"review": review})

# 결과 출력
print("\n===== 여행 후기 분석 결과 =====")
pprint(output)


출력 형식 지시사항:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"destination": string  // 여행한 장소 또는 도시명
	"duration": string  // 여행 기간 (예: '2박 3일')
	"budget": string  // 전체 여행 예산 또는 비용 (예: '30만원')
	"rating": string  // 5점 만점에서 추천도 (숫자 1~5 중 하나)
	"activities": string  // 주요 활동 목록 (문자열 리스트)
}
```

===== 여행 후기 분석 결과 =====
{'activities': '해운대에서 바다구경, 자갈치시장에서 회 먹기, 감천문화마을 구경',
 'budget': '100만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4'}
